In [1]:
#dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from matplotlib.lines import Line2D

In [2]:
path = '../statesAbbrev.csv'
statesAbbrev_df = pd.read_csv(path)
statesAbbrev_df.head(20)

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA
5,Colorado,Colo.,CO
6,Connecticut,Conn.,CT
7,Delaware,Del.,DE
8,District of Columbia,D.C.,DC
9,Florida,Fla.,FL


In [3]:
#list for state codes
state_codes = [x for x in statesAbbrev_df['Code']]

In [4]:
#list for state names
state_names = [x for x in statesAbbrev_df['State']]

In [5]:
#creating a df dict to hold the state dfs, reading in the dfs
states_df_dict = {}
for x in range(10, 20):
    path = f'../states_data/US-{state_codes[x]}_cat_data.csv'
    states_df_dict[f'{state_codes[x]}_data_df'] = [pd.read_csv(path)]

In [6]:
#inspecting the keys (ie df names)
states_df_dict.keys()

dict_keys(['GA_data_df', 'HI_data_df', 'ID_data_df', 'IL_data_df', 'IN_data_df', 'IA_data_df', 'KS_data_df', 'KY_data_df', 'LA_data_df', 'ME_data_df'])

In [7]:
#test printing a df
print(state_codes[10])
states_df_dict[f'{state_codes[10]}_data_df'][0].head(10)

GA


,time,arts_entertainment,autos,beauty_fitness,books_lit,action_adventure,campaigns_elections,celebs,discrimination,entertainment_media,...,mobiles,online_vids,scifi_fantasy,sport_news,tv_shows,voice_vid_chat,weather,covid_cases,stay_at_home,mass_gathering_ban
0,2019-01-06,85,68,83,72,57,0,61,15,83,...,55,77,63,76,82,65,51,NaN,False,False
1,2019-01-13,85,71,88,85,72,0,61,17,85,...,57,85,48,67,85,48,64,NaN,False,False
2,2019-01-20,85,73,85,82,64,0,56,16,81,...,57,91,61,69,84,64,69,NaN,False,False
3,2019-01-27,89,75,87,85,63,0,69,22,75,...,59,90,53,70,85,79,100,NaN,False,False
4,2019-02-03,83,73,83,80,59,0,56,20,69,...,54,79,46,85,84,65,54,NaN,False,False
5,2019-02-10,80,69,78,78,54,0,54,25,70,...,55,78,48,52,79,63,63,NaN,False,False
6,2019-02-17,84,72,84,72,60,0,72,19,79,...,60,80,50,52,79,63,74,NaN,False,False
7,2019-02-24,87,77,85,84,57,0,60,25,70,...,57,77,56,54,81,77,63,NaN,False,False
8,2019-03-03,86,76,84,83,56,0,52,42,66,...,58,74,48,54,94,70,78,NaN,False,False
9,2019-03-10,81,74,82,71,53,0,53,15,63,...,54,80,46,61,80,60,56,NaN,False,False


In [8]:
col_names = list(states_df_dict[f'{state_codes[10]}_data_df'][0].columns)

In [9]:
col_names_for_max = [x for x in col_names if x not in ['time', 'covid_cases', 'stay_at_home', 'mass_gathering_ban']]

In [11]:
times_list = list(states_df_dict[f'{state_codes[10]}_data_df'][0]['time'])

In [12]:

for x in range(10, 20):
    for y in range(1, len(states_df_dict[f'{state_codes[x]}_data_df'])):
        del(states_df_dict[f'{state_codes[x]}_data_df'][y])
    print(state_codes[x])
    print(len(states_df_dict[f'{state_codes[x]}_data_df']))

GA
1
HI
1
ID
1
IL
1
IN
1
IA
1
KS
1
KY
1
LA
1
ME
1


In [13]:
def means_comp(ind_no):
    all_data_df = states_df_dict[f'{state_codes[ind_no]}_data_df'][0]
    data_2019_df = all_data_df.iloc[:52, :]
    data_2020_df = all_data_df.iloc[52:, :]
    
    means_diff = []
    for col in col_names_for_max:
        means_diff_instance = stats.ttest_ind(data_2019_df[col], data_2020_df[col], equal_var=False)[1]
        means_diff.append(means_diff_instance)
    means_diff[21:24] = 'n/a', 'n/a', 'n/a'
    
    signif_list = []
    for x in means_diff:
        if x == 'n/a':
            signif_list.append('n/a')
        elif x <= 0.05:
            signif_list.append('Yes')
        elif x > 0.05:
            signif_list.append('No')

    comparison_df = pd.DataFrame({'categories': data_2019_df.mean().index, '2019_means': data_2019_df.mean().values, '2020_means': data_2020_df.mean().values, 'p_vals': means_diff, 'stats_signif': signif_list})
    comparison_df.drop([21,22,23], inplace=True)
    comparison_df.set_index('categories', inplace=True)
    new_index = ['action_adventure', 'scifi_fantasy', 'tv_shows', 'online_vids', 'mobiles', 'voice_vid_chat', 'infectious_diseases', 'health_news', 'games_systems_consoles', 'arts_entertainment', 'books_lit', 'lottos', 'celebs', 'entertainment_media', 'campaigns_elections', 'discrimination', 'law_enf', 'weather', 'autos', 'beauty_fitness', 'sport_news']
    comparison_df = comparison_df.reindex(new_index)
    states_df_dict[f'{state_codes[ind_no]}_data_df'].append(comparison_df)
    
    signif_cats =[]
    for x in range(0, len(comparison_df)):
        if comparison_df.iloc[x, 3] == 'Yes':
            signif_cats.append((list(comparison_df.index)[x], comparison_df.iloc[x, 2]))
    states_df_dict[f'{state_codes[ind_no]}_data_df'].append(signif_cats)

In [14]:
for x in range(10, 20):
    means_comp(x)

In [16]:
print(state_codes[10])
means_meds_df = states_df_dict[f'{state_codes[10]}_data_df'][1]
means_meds_df

GA


,2019_means,2020_means,p_vals,stats_signif
categories,,,,
action_adventure,64.250000,67.784314,0.0307654,Yes
scifi_fantasy,55.115385,61.215686,0.0213611,Yes
tv_shows,82.788462,82.901961,0.91907,No
online_vids,80.326923,77.392157,0.0288844,Yes
mobiles,59.615385,58.156863,0.29653,No
voice_vid_chat,60.846154,55.215686,0.0274913,Yes
infectious_diseases,8.903846,23.921569,2.68146e-06,Yes
health_news,54.884615,67.960784,1.16964e-07,Yes
games_systems_consoles,43.384615,55.352941,2.09833e-07,Yes


In [17]:
states_df_dict[f'{state_codes[10]}_data_df'][2]

[('action_adventure', 0.030765368418572834),
 ('scifi_fantasy', 0.021361148900173967),
 ('online_vids', 0.028884448751748344),
 ('voice_vid_chat', 0.027491300944616104),
 ('infectious_diseases', 2.681462180525121e-06),
 ('health_news', 1.1696353558554621e-07),
 ('games_systems_consoles', 2.098326850605566e-07),
 ('lottos', 0.002668846338999254),
 ('celebs', 2.2602375695717654e-05),
 ('entertainment_media', 0.011456932860112484),
 ('discrimination', 0.0010632797599327826),
 ('beauty_fitness', 0.0021862257810211063),
 ('sport_news', 2.380168950013029e-08)]

In [18]:
for x in range(10, 20):
    print(state_codes[x])
    print(states_df_dict[f'{state_codes[x]}_data_df'][2])


GA
[('action_adventure', 0.030765368418572834), ('scifi_fantasy', 0.021361148900173967), ('online_vids', 0.028884448751748344), ('voice_vid_chat', 0.027491300944616104), ('infectious_diseases', 2.681462180525121e-06), ('health_news', 1.1696353558554621e-07), ('games_systems_consoles', 2.098326850605566e-07), ('lottos', 0.002668846338999254), ('celebs', 2.2602375695717654e-05), ('entertainment_media', 0.011456932860112484), ('discrimination', 0.0010632797599327826), ('beauty_fitness', 0.0021862257810211063), ('sport_news', 2.380168950013029e-08)]
HI
[('scifi_fantasy', 0.006137448531335592), ('tv_shows', 0.021530077306697686), ('online_vids', 0.002701019526670935), ('mobiles', 0.006941725491945956), ('infectious_diseases', 4.148211200708046e-07), ('games_systems_consoles', 9.256178552037575e-05), ('arts_entertainment', 2.1784499911431002e-06), ('books_lit', 0.014899380751265085), ('lottos', 5.075585217997051e-06), ('discrimination', 0.04059018753214644), ('law_enf', 0.0020948504737188533

In [19]:
states_df_dict[f'{state_codes[10]}_data_df'][0].describe()

,arts_entertainment,autos,beauty_fitness,books_lit,action_adventure,campaigns_elections,celebs,discrimination,entertainment_media,games_systems_consoles,...,law_enf,lottos,mobiles,online_vids,scifi_fantasy,sport_news,tv_shows,voice_vid_chat,weather,covid_cases
count,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.00000,...,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,42.000000
mean,83.776699,78.155340,84.951456,70.990291,66.000000,1.621359,56.252427,20.048544,76.368932,49.31068,...,44.233010,75.233010,58.893204,78.873786,58.135922,62.941748,82.844660,58.058252,49.708738,193311.952381
std,4.283878,8.559775,6.475260,11.479304,8.343131,9.882760,9.937766,11.877957,8.909117,12.06033,...,9.523144,6.523267,7.084798,6.819349,13.547338,14.871888,5.640813,12.999114,14.533687,160818.998176
min,71.000000,61.000000,67.000000,49.000000,52.000000,0.000000,40.000000,7.000000,53.000000,32.00000,...,32.000000,64.000000,48.000000,64.000000,38.000000,34.000000,68.000000,30.000000,21.000000,7.000000
25%,81.000000,71.000000,80.000000,61.000000,60.000000,0.000000,51.000000,14.000000,70.000000,40.00000,...,40.000000,71.000000,55.000000,74.000000,48.000000,54.000000,79.000000,49.000000,41.500000,38985.250000
50%,84.000000,76.000000,84.000000,72.000000,65.000000,0.000000,55.000000,17.000000,76.000000,46.00000,...,42.000000,75.000000,57.000000,78.000000,55.000000,64.000000,83.000000,57.000000,47.000000,180565.000000
75%,86.000000,84.000000,89.000000,78.000000,70.500000,1.000000,59.000000,21.000000,82.000000,54.00000,...,45.000000,78.500000,60.500000,83.000000,64.500000,74.000000,86.500000,63.500000,60.000000,329288.000000
max,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,509588.000000
